In [ ]:
import pandas as pd

df = pd.read_csv('xy.csv')

# =============================================================================
# 1. CAGR 계산 (판다랭크 데이터 기반)
# =============================================================================
# keyword_search_3y.csv 사용 (행: 날짜, 열: 행정동)
panda = pd.read_csv('keyword_search_3y.csv', index_col=0)
panda.index = pd.to_datetime(panda.index)

# 연도별 합계 계산
panda['year'] = panda.index.year
yearly_sum = panda.groupby('year').sum()

# 행정동별 CAGR 계산
cagr = {}
for col in yearly_sum.columns:
    if col == 'year':
        continue
    sum_2023 = yearly_sum.loc[2023, col]
    sum_2025 = yearly_sum.loc[2025, col]
    cagr[col] = (sum_2025 / sum_2023) ** (1/2) - 1 if sum_2023 > 0 else 0

# 결과
df['CAGR'] = df['행정동'].map(cagr)

In [20]:
# =============================================================================
# 2-1. 네이버 검색률 평균 계산
# =============================================================================
# naver_trend.csv 사용 (행: 날짜, 열: 행정동)
naver = pd.read_csv('naver_trend.csv', index_col=0)
naver.index = pd.to_datetime(naver.index)

# 행정동별 전체 평균 계산
avg_naver = {}
for col in naver.columns:
    avg_naver[col] = naver[col].mean()

df['avg_naver'] = df['행정동'].map(avg_naver)

# =============================================================================
# 2-2. 네이버 성장률 계산
# =============================================================================
# naver_trend.csv 사용 (행: 날짜, 열: 행정동)

# 행정동별 성장률 계산
naver_growth = {}
for col in naver.columns:
    first_3m = naver[col].head(90).mean()   # 처음 90일 평균
    last_3m = naver[col].tail(90).mean()    # 최근 90일 평균
    naver_growth[col] = (last_3m / first_3m) - 1

# 결과
df['naver_growth'] = df['행정동'].map(naver_growth)



In [21]:
# =============================================================================
# 3-1. 블로그 총합
# =============================================================================
# blog.csv 사용 (행: 전체게시글, 열: 행정동)

# 3. 블로그 총합 계산
blog = pd.read_csv('blog.csv', index_col=0)

blog_post = {}
for col in blog.columns:
    total = float(str(blog.loc['전체 게시글', col]).replace(',', ''))
    blog_post[col] = total

df['blog_post'] = df['행정동'].map(blog_post)

# 4. 확인
print(df[['행정동', 'avg_naver', 'naver_growth', 'blog_post']])


# =============================================================================
# 3-2. 블로그 신선도 계산
# =============================================================================
# blog.csv 사용 (행: 월 발행량, 전체 게시글, 열: 행정동)

# 3. 블로그 신선도 계산

blog_freshness = {}
for col in blog.columns:
    monthly = float(str(blog.loc['월 발행량', col]).replace(',', ''))
    total = float(str(blog.loc['전체 게시글', col]).replace(',', ''))
    blog_freshness[col] = monthly / total if total > 0 else 0

df['blog_freshness'] = df['행정동'].map(blog_freshness)

# 4. 확인
print(df[['행정동', 'avg_naver', 'naver_growth', 'blog_freshness']])

     행정동  avg_naver  naver_growth  blog_post
0    가산동   3.704718     -0.403844   131240.0
1   구로3동   3.494759     -0.117110   186588.0
2   논현2동   3.421647     -0.374366    37964.0
3   대치4동   5.276750     -0.367127   177561.0
4   문정2동   2.930910     -0.198994   103715.0
5   방배1동   2.298677     -0.439810   104467.0
6   삼성1동  18.274635     -0.202306   320376.0
7   삼성2동   2.750442     -0.278754    43155.0
8    상암동   3.633894     -0.400226   134958.0
9    서교동  33.240818     -0.437335   181063.0
10  서초2동   3.632072     -0.671689   575268.0
11  서초3동  27.991549     -0.522919   125476.0
12   성수동  36.105341      1.413849  1013957.0
13  송파2동   0.904031     -0.198465   354804.0
14   신당동   1.916366      0.435393   193850.0
15  압구정동  15.449557     -0.353047   910140.0
16  양평2동   4.176988     -0.174195    91502.0
17   여의동  20.352348      0.334175   361290.0
18   염리동   3.656472     -0.148187    80461.0
19   용강동   1.724418     -0.340532   358838.0
20   중림동   0.563528     -0.244106    54989.0
21   청담동  

In [22]:
# =============================================================================
# 4. 유튜브 신선도 계산
# =============================================================================
from datetime import timedelta

print("\n🎬 유튜브 신선도 계산 중...")
YOUTUBE_FILE = 'youtube_data.csv'

yt = pd.read_csv(YOUTUBE_FILE, encoding='utf-8-sig')
yt['publishedAt'] = pd.to_datetime(yt['publishedAt'])

# 기준일 설정 (최근 1년)
max_date = yt['publishedAt'].max()
cutoff = max_date - timedelta(days=365)  # ← 이거 추가!

yt_freshness = {}
for district in yt['district'].unique():
    district_data = yt[yt['district'] == district]
    recent = district_data[district_data['publishedAt'] >= cutoff]
    yt_freshness[district] = len(recent) / len(district_data)  # 0~1

df['yt_freshness'] = df['행정동'].map(yt_freshness)

# 유튜브 데이터 없는 행정동은 0으로
df['yt_freshness'] = df['yt_freshness'].fillna(0)
print(f"   ✅ 완료!")


🎬 유튜브 신선도 계산 중...
   ✅ 완료!


In [ ]:
print("\n" + "=" * 80)
print("📊 최종 결과")
print("=" * 80)
print(f"{'행정동':<10} {'CAGR':>8} {'avg_naver':>10} {'naver_growth':>14} {'blog_post':>12} {'blog_freshness':>16} {'yt_freshness':>14}")
print("-" * 80)


df['naver_growth'] = df['naver_growth'].round(9)
df['blog_freshness'] = df['blog_freshness'].round(9)
df['yt_freshness'] = df['yt_freshness'].round(9)


for _, row in df.iterrows():
    print(f"{row['행정동']:<10} {row['CAGR']:>8.2f} {row['avg_naver']:>10.2f} {row['naver_growth']:>14.3f} {row['blog_post']:>12,} {row['blog_freshness']:>16.4f} {row['yt_freshness']:>14.4f}")

# =============================================================================
# 5. 저장
# =============================================================================


OUTPUT_FILE = 'final_xy_updated.csv' 
df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"\n✅ 저장 완료: {OUTPUT_FILE}")
print(f"추가된 컬럼: CAGR, avg_naver, naver_growth, blog_post, blog_freshness, yt_freshness")


📊 최종 결과
행정동          cagr_2  avg_naver   naver_growth    blog_post   blog_freshness   yt_freshness
--------------------------------------------------------------------------------
가산동           -0.10       3.70         -0.404    131,240.0           0.0164         0.4900
구로3동          -0.01       3.49         -0.117    186,588.0           0.0155         0.6100
논현2동          -0.05       3.42         -0.374     37,964.0           0.0315         0.4500
대치4동          -0.03       5.28         -0.367    177,561.0           0.0384         0.5200
문정2동           0.01       2.93         -0.199    103,715.0           0.0256         0.6300
방배1동          -0.06       2.30         -0.440    104,467.0           0.0251         0.5500
삼성1동          -0.13      18.27         -0.202    320,376.0           0.0230         0.3400
삼성2동          -0.05       2.75         -0.279     43,155.0           0.0649         0.6200
상암동           -0.08       3.63         -0.400    134,958.0           0.0168         0.5200
